# Algorithme MCMC

Une fois obtenu le MLE, nous utilisons un un algorithme de Monte-Carlo pour affiner le résultat : 
1) On pose $Z_0 = Z_{MLE}$ et $\alpha_0 \sim Exp(\lambda)$
2) On tire $\overset{-}{Z}_{k+1} \sim \mathcal{N}(Z_k, \sigma^2 I_{n.k})$ ($n$ taille de Z, $k$ dimension des points)
3) On réalise la transformation procustéenne : $\overset{\sim}{Z} = \underset{T}{argmin}\ tr \left[\left(
Z_0-T \overset{-}{Z}_{k+1}\right)^t \left( Z_0-T \overset{-}{Z}_{k+1}\right) \right] $ (T est une opération combinant la translation, la rotation et la symétrie).
4) On accepte $\overset{\sim}{Z}_{k+1}$ en tant  que $Z_{k+1}$ avec probilité $max \left\{1, \frac{\mathbb{P}(Y|\overset{\sim}{Z}_{k+1}, \alpha)}{\mathbb{P}(Y|Z_k, \alpha)} \frac{\pi_1(\overset{\sim}{Z})}{\pi_1(Z_k)} \right\}$, sinon on pose $Z_{k+1}=Z_k$.
5) On tire $\overset{\sim}{\alpha} \sim Exp(\lambda)$ (on ne peut tirer selon une loi normale centrée en $\alpha$ au risque d'avoir un $\alpha$ négatif.)
6) On accepte $\overset{\sim}{\alpha}$ en tant  que $\alpha_{k+1}$ avec probilité $max \left\{1, \frac{\mathbb{P}(Y|Z_{k+1}, \overset{\sim}{\alpha)}}{\mathbb{P}(Y|Z_{k+1}, \alpha)} \frac{\pi_2(\overset{\sim}{\alpha})}{\pi_2(\alpha_k)} \right\}$, sinon on pose $\alpha_{k+1}=\alpha_k$

_L'étape 3 (la transformation procustéenne) est nécessaire car notre modèle ne tient compte que des distances. Donc pour un nuage de point donnée, tout nuage de point obtenu par translation, rotation ou symétrie de ce nuage de point donne les mêmes résultats. Comme on veut s'assurer de la bonne convergence du modèle, on force les nouveaux points obtenus à être le plus proche du nuage de point initial via ces transformations. Cependant, on ne fait pas de mise à l'échelle contrairement à ce qui peut se faire dans d'autres transformations procustéenne, car on modifierait alors la distance._

On prendra pour loi $\pi_1$ une loi normale centrée, de variance suffisament grande, et pour loi $\pi_2$ une loi normale exponentielle, de paramètre 2, ce paramètre permettant de ne pas d'avoir d'$\alpha$ trop petit ou trop grand.

On importe les bibliothèques et les variables nécessaires :

In [3]:
!py -m pip install numpy
!py -m pip install scipy
import numpy as np
from scipy.linalg import orthogonal_procrustes

from MLE import MLE

np.random.seed(0)

data = np.loadtxt("data/Florentine_families.csv", delimiter=",")
n = np.shape(data)[0]  # = 15

DIMENSIONS = 2
SIGMA = 100
LAMBDA = 2


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


On définit les lois $\pi_1$ et $\pi_2$ :

In [6]:
def pi_1(Z: np.matrix, sigma: float = SIGMA) -> float:
    """
    Renvoie le Log de la loi a priori de Z (a une constante pres)
    """
    return -np.linalg.norm(Z) / (2 * sigma**2)


def pi_2(alpha: float, lambd: float = LAMBDA) -> float:
    """
    Renvoie le Log de la loi a priori de alpha (a une constante pres)
    """
    return -lambd * alpha

On définie le log de la probabilité d'acceptation :

In [7]:
def test_Z(
    alpha: float, Z_tilde: np.matrix, Z: np.matrix, Y: np.matrix, sigma: float = SIGMA
) -> float:
    """
    Renvoie la probabilite d'acceptation pour Z
    """
    log_prob = 0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            eta = alpha - np.linalg.norm(Z[i] - Z[j])
            eta_tilde = alpha - np.linalg.norm(Z_tilde[i] - Z_tilde[j])
            log_prob += (
                Y[i][j] * eta_tilde
                - np.log(1 + np.exp(eta_tilde))
                - Y[i][j] * eta
                - np.log(1 + np.exp(eta))
            )
        log_prob += pi_1(Z_tilde, sigma) - pi_1(Z, sigma)
    return np.exp(log_prob)

def test_alpha(
    alpha_tilde: float, alpha: float, Z: np.matrix, Y: np.matrix, lambd: float = LAMBDA
) -> float:
    """
    Renvoie la probabilite d'acceptation pour alpha
    """
    log_prob = 0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            eta = alpha - np.linalg.norm(Z[i] - Z[j])
            eta_tilde = alpha_tilde - np.linalg.norm(Z[i] - Z[j])
            log_prob += (
                Y[i][j] * eta_tilde
                - np.log(1 + np.exp(eta_tilde))
                - Y[i][j] * eta
                + np.log(1 + np.exp(eta))
            )
    log_prob += pi_2(alpha_tilde, lambd) - pi_2(alpha, lambd)
    return np.exp(log_prob)

On définie une fonction réalisant une étape de l'algorithme de Monte Carlo :

In [8]:
def MCMC(
    alpha: float,
    Z: np.matrix,
    Y: np.matrix,
    pas1: float = 1,
    pas2: float = 0.5,
    lambd: float = LAMBDA,
    sigma: float = SIGMA,
) -> tuple[float, np.matrix]:
    """
    Cette fonction prend un couple (alpha,Z), et renvoie un nouveau couple (alpha,Z).
    Le nouveau Z est tire selon une loi normale centree en Z, de variance pas1**2. Puis accepte, avec pour prior
    la loi normale centree, de variance sigma**2.
    Le nouveau alpha est tire selon une loi normale centree en Z, de variance pas2**2. Puis accepte, avec pour prior
    la loi exponentielle de parametre lambda.
    """
    Z_tilde = Z + np.random.normal(0, pas1, size=Z.shape)
    Z_tilde = Z_tilde - np.mean(Z_tilde, axis=0)
    Omega, _ = orthogonal_procrustes(Z, Z_tilde)

    Z_tilde = np.dot(Z_tilde, Omega)
    alpha_tilde = abs(alpha + np.random.normal(0, pas2))

    prob_Z = test_Z(alpha, Z_tilde, Z, Y, sigma)
    if np.random.random() < prob_Z:
        Z = Z_tilde

    prob_alpha = test_alpha(alpha, alpha, Z, Y, lambd)  # = tout le temps 0 ??
    if np.random.random() < prob_alpha:
        alpha = alpha_tilde
    return alpha, Z